<a href="https://colab.research.google.com/github/krakowiakpawel9/convnet-course/blob/master/04_mnist_transfer_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Transfer Learning

In [0]:
import time
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


### Zdefiniowanie stałych parametrów

In [0]:
batch_size = 128
num_classes = 5
epochs = 5
filters=32
kernel_size=3
pool_size=2

# rozmiar obrazu wejsciowego
img_rows, img_cols = 28, 28

In [0]:
if K.image_data_format() == 'channel_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)
print(input_shape)    

(28, 28, 1)


### Pobranie danych

In [0]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

### Budowa modelu

In [0]:
def train_model(model, train, test, num_classes):
    train_images = train[0].reshape((train[0].shape[0],) + input_shape)
    test_images = test[0].reshape((test[0].shape[0],) + input_shape)
    train_images = train_images.astype('float32')
    test_images = test_images.astype('float32')
    train_images /= 255
    test_images /= 255
    print('train_images shape:', train_images.shape)
    print(train_images.shape[0], 'train samples')
    print(test_images.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    train_labels = keras.utils.to_categorical(train[1], num_classes)
    test_labels = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    
    model.fit(train_images, train_labels,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(test_images, test_labels))

    score = model.evaluate(test_images, test_labels, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

### Tworzymy dwa zestawy danych, jeden z cyframi poniżej 5 i drugi powyżej 5

In [0]:
train_images_lt5 = train_images[train_labels < 5]
train_labels_lt5 = train_labels[train_labels < 5]
test_images_lt5 = test_images[test_labels < 5]
test_labels_lt5 = test_labels[test_labels < 5]

train_images_gte5 = train_images[train_labels >= 5]
train_labels_gte5 = train_labels[train_labels >= 5] - 5
test_images_gte5 = test_images[test_labels >= 5]
test_labels_gte5 = test_labels[test_labels >= 5] - 5

### Zdefiniowanie dwóch grup warstw: warstwa cech i warstwa klasyfikacji

In [0]:
feature_layers = [
    Conv2D(filters, kernel_size, padding='valid', input_shape=input_shape, activation='relu'),
    Conv2D(filters, kernel_size, activation='relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
]

# skompletowanie modelu
model = Sequential(feature_layers + classification_layers)

W0711 11:39:29.109920 140705214871424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 11:39:29.127689 140705214871424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 11:39:29.130934 140705214871424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 11:39:29.165940 140705214871424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0711 11:39:29.169354 140705214871424 deprecation_wrapp

### Trenowanie modelu klasyfikacji [0, 1, 2, 3, 4]

In [0]:
train_model(model,
            train=(train_images_lt5, train_labels_lt5),
            test=(test_images_lt5, test_labels_lt5),
            num_classes=num_classes)

W0711 11:39:47.063200 140705214871424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0711 11:39:47.084811 140705214871424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0711 11:39:47.196398 140705214871424 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


train_images shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 6s 183us/step - loss: 0.1590 - acc: 0.9483 - val_loss: 0.0289 - val_acc: 0.9907
Epoch 2/5
30596/30596 [==============================] - 3s 109us/step - loss: 0.0434 - acc: 0.9875 - val_loss: 0.0135 - val_acc: 0.9953
Epoch 3/5
30596/30596 [==============================] - 4s 116us/step - loss: 0.0297 - acc: 0.9904 - val_loss: 0.0072 - val_acc: 0.9979
Epoch 4/5
30596/30596 [==============================] - 4s 115us/step - loss: 0.0226 - acc: 0.9932 - val_loss: 0.0077 - val_acc: 0.9981
Epoch 5/5
30596/30596 [==============================] - 4s 115us/step - loss: 0.0189 - acc: 0.9937 - val_loss: 0.0057 - val_acc: 0.9982
Test score: 0.00569881150620759
Test accuracy: 0.9982486865148862


### Wykorzystanie transfer learning

In [0]:
# zamrożenie warstwy cech i przebudowa modelu
for l in feature_layers:
    l.trainable = False

for i in model.layers:
    print('Warstwa trenowana:',i.trainable)

Warstwa trenowana: False
Warstwa trenowana: False
Warstwa trenowana: False
Warstwa trenowana: False
Warstwa trenowana: False
Warstwa trenowana: True
Warstwa trenowana: True
Warstwa trenowana: True


### transfer: trenowanie dla nowego zadania klasyfikacji [5, 6, 7, 8, 9]

In [0]:
train_model(model,
            train=(train_images_gte5, train_labels_gte5),
            test=(test_images_gte5, test_labels_gte5),
            num_classes=num_classes)

train_images shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 3s 88us/step - loss: 0.2367 - acc: 0.9338 - val_loss: 0.0506 - val_acc: 0.9825
Epoch 2/5
29404/29404 [==============================] - 2s 73us/step - loss: 0.0738 - acc: 0.9767 - val_loss: 0.0369 - val_acc: 0.9877
Epoch 3/5
29404/29404 [==============================] - 2s 73us/step - loss: 0.0558 - acc: 0.9831 - val_loss: 0.0276 - val_acc: 0.9901
Epoch 4/5
29404/29404 [==============================] - 2s 70us/step - loss: 0.0480 - acc: 0.9851 - val_loss: 0.0279 - val_acc: 0.9905
Epoch 5/5
29404/29404 [==============================] - 2s 70us/step - loss: 0.0433 - acc: 0.9868 - val_loss: 0.0241 - val_acc: 0.9928
Test score: 0.024140555971219755
Test accuracy: 0.9927998354248098
